# West Gulf Coast Mottled Duck Nesting
This is the 2023 update to this 2022 indicator for the 2022 base blueprint. This update includes added a 0 value for the analsyis extent. 

Created by Blair Tirpak
Updated by Blair Tirpak on March 22, 2023
Last run by Blair Tirpak on August 29, 2023 - updated color scheme

In [1]:
# Import system modules
import arcpy
from arcpy import env
from arcpy.sa import *

In [2]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace = 
env.workspace = r"E:\Blueprint-SE\2023\MODU\MODUgrasslands.gdb"
# extent
SEraster = r"E:\Blueprint-SE\2023\MODU\BaseBlueprintExtent2022.tif"
# snap raster 
arcpy.env.snapRaster = SEraster
arcpy.env.cellSize = SEraster

In [3]:
# define inupts
grass = r"E:\Blueprint-SE\2023\MODU\JV\MODU.gdb\CurrentlySuitableNesting"
extent =  r"E:\Blueprint-SE\2023\MODU\JV\MODU_range_byInitiaveArea.shp"

In [4]:
# define final outputs
SEout = r"E:\Blueprint-SE\2023\MODU\WestGulfCoastMottledDuckNesting.tif"

In [5]:
arcpy.management.ProjectRaster(grass, "grassland", 'PROJCS["NAD_1983_Contiguous_USA_Albers",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-96.0],PARAMETER["Standard_Parallel_1",29.5],PARAMETER["Standard_Parallel_2",45.5],PARAMETER["Latitude_Of_Origin",23.0],UNIT["Meter",1.0]]', "BILINEAR", "30 30", "WGS_1984_(ITRF00)_To_NAD_1983", None, 'PROJCS["WGS_1984_UTM_Zone_15N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', "NO_VERTICAL")

<Result 'E:\\Blueprint-SE\\2023\\MODU\\MODUgrasslands.gdb\\grassland'>

### Start Analysis

In [6]:
# Print the current workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

E:\Blueprint-SE\2023\MODU\MODUgrasslands.gdb


## Slice grassland layer into 10 equal area classes. 

In [7]:
# slice into 10 classes
outSlice = Slice("grassland", 10, "EQUAL_AREA", 1)
outSlice.save("MODU_10cl")

## Create extent raster

In [8]:
# project extent file
arcpy.management.Project(extent, "MODU_range", 'PROJCS["WGS_1984_UTM_Zone_15N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', None, 'PROJCS["WGS_1984_UTM_Zone_15N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")
# disolve poly    
arcpy.analysis.PairwiseDissolve("MODU_range", "MODU_range_dis", None, None, "MULTI_PART")    

<Result 'E:\\Blueprint-SE\\2023\\MODU\\MODUgrasslands.gdb\\MODU_range_dis'>

In [11]:
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    arcpy.conversion.PolygonToRaster("MODU_range_dis", "OBJECTID", "MODU_range_R", "CELL_CENTER", "NONE", r"E:\Blueprint-SE\2022\2022Indicators\BBP_extent\BaseBlueprintExtent2022.tif", "BUILD")

In [12]:
remap = RemapValue([[1, 0]])

# Execute Reclassify
Range0 = Reclassify("MODU_range_R", "Value", remap, "NODATA")
Range0.save("Range0")

In [22]:
arcpy.MosaicToNewRaster_management("MODU_10cl;Range0", env.workspace,"MODU_11cl", sr,\
                                   "8_BIT_UNSIGNED", 30, 1, "MAXIMUM", "FIRST")

<Result 'E:\\Blueprint-SE\\2023\\MODU\\MODUgrasslands.gdb\\MODU_11cl'>

In [23]:
# clip slice to SE 2022 Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.ExtractByMask("MODU_11cl", SEraster); out_raster.save("MODU_11cl_clip")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [11]:
# set code block for next step

codeblock = """
def Reclass(v):
	if v == 10:
		return '10 = 90th-100th percentile of suitable mottled duck nesting habitat'
	elif v == 9:
		return '9 = 80th-90th percentile of suitable mottled duck nesting habitat'
	elif v == 8:
		return '8 = 70th-80th percentile of suitable mottled duck nesting habitat'
	elif v == 7:
		return '7 = 60th-70th percentile of suitable mottled duck nesting habitat'    
	elif v == 6:
		return '6 = 50th-60th percentile of suitable mottled duck nesting habitat' 
	elif v == 5:
		return '5 = 40th-50th percentile of suitable mottled duck nesting habitat'   
	elif v == 4:
		return '4 = 30th–40th percentile of suitable mottled duck nesting habitat'   
	elif v == 3:
		return '3 = 20th-30th percentile of suitable mottled duck nesting habitat'   
	elif v == 2:
		return '2 = 10th-20th percentile of suitable mottled duck nesting habitat'   
	elif v == 1:
		return '1 = 0-10th percentile of suitable mottled duck nesting habitat'   
	elif v == 0:
		return '0 = Not identified as suitable mottled duck nesting habitat (within TX and LA)'        
"""

In [12]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("MODU_11cl_clip", "descript", "Reclass(!Value!)", "PYTHON3", codeblock, "TEXT")

<Result 'MODU_11cl_clip'>

### Code Color scheme

In [6]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 10:
		return 128
	if Value == 9:
		return 182
	if Value == 8:
		return 219
	if Value == 7:
		return 244
	if Value == 6:
		return 253
	if Value == 5:
		return 253
	if Value == 4:
		return 254
	if Value == 3:
		return 254
	if Value == 2:
		return 255
	if Value == 1:
		return 255
	if Value == 0:
		return 255      
	else:
		return 255
		
def Reclass2(Value):
	if Value == 10:
		return 0
	if Value == 9:
		return 0
	if Value == 8:
		return 20
	if Value == 7:
		return 61
	if Value == 6:
		return 113
	if Value == 5:
		return 157
	if Value == 4:
		return 191
	if Value == 3:
		return 221
	if Value == 2:
		return 239
	if Value == 1:
		return 255
	if Value == 0:
		return 255
	else:
		return 255
		
def Reclass3(Value):
	if Value == 10:
		return 38
	if Value == 9:
		return 38
	if Value == 8:
		return 30
	if Value == 7:
		return 37
	if Value == 6:
		return 52
	if Value == 5:
		return 67
	if Value == 4:
		return 90
	if Value == 3:
		return 127
	if Value == 2:
		return 163
	if Value == 1:
		return 190
	if Value == 0:
		return 255
	else:
		return 255
"""

In [7]:
# calculate Red field
arcpy.management.CalculateField("MODU_11cl_clip", "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField("MODU_11cl_clip", "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField("MODU_11cl_clip", "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'MODU_11cl_clip'>

In [8]:
# export as .tif with SE extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    arcpy.management.CopyRaster("MODU_11cl_clip", SEout, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

## FIN